In [1]:
specifics = 'calculated_limb_calculated_alpha'
path = "current_fits"

In [3]:
from automatic_calculated_limb_calculated_alpha import *
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from math import *
from datetime import datetime

In [4]:
sweet_cat_parameters = pd.read_csv("all_spectra_parameters.csv")
sweet_cat_parameters.astype({'vmic': 'float', 'err_vmic': 'float'}).dtypes
sweet_cat_parameters

,ind,Spectra,SNR,Resolution,Star,Star_SC,teff,err_teff,logg,err_logg,vmic,err_vmic,MH,err_MH,vsini,err_up_vsini,err_down_vsini
0,0,HD19994A_HARPSS_115000_378_691_2020.fits,2021.85,115000,HD19994A,HD 19994 A,6249,27,4.26,0.04,1.57,0.03,0.27,0.02,8.60,1.720,-1.720
1,1,HD60532_HARPSS_115000_378_691_2020.fits,1940.86,115000,HD60532,HD 60532,6281,34,4.00,0.04,1.80,0.04,-0.07,0.02,8.00,1.600,-1.600
2,2,HR858_HARPSS_115000_378_691_2020.fits,1772.89,115000,HR858,HR 858,6360,26,4.50,0.04,1.51,0.04,-0.02,0.02,8.30,0.500,-0.500
3,3,HD10647_HARPSS_115000_378_691_2020.fits,1696.77,115000,HD10647,HD 10647,6178,20,4.49,0.03,1.26,0.03,-0.01,0.01,4.90,0.980,-0.980
4,4,HD208487_HARPSS_115000_378_691_2020.fits,1461.86,115000,HD208487,HD 208487,6137,18,4.46,0.03,1.21,0.03,0.09,0.01,4.61,0.922,-0.922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,90,WASP-135_SOPHIE_HE_2020.fits,82.53,40000,WASP-135,WASP-135,5052,77,4.11,0.22,0.85,0.14,0.17,0.05,4.67,0.890,-0.890
91,91,WASP-139_HARPSS_115000_378_691_2020.fits,56.31,115000,WASP-139,WASP-139,5215,38,4.35,0.08,0.82,0.07,0.12,0.03,4.20,1.100,-1.100
92,92,WASP-150_SOPHIE_HE_2020.fits,127.76,40000,WASP-150,WASP-150,6486,48,4.46,0.09,1.61,0.06,0.27,0.03,8.82,0.950,-0.950
93,93,WASP-173A_HARPSS_115000_378_691_2020.fits,110.24,115000,WASP-173A,KELT-22A,5803,26,4.45,0.04,1.29,0.03,0.21,0.02,6.10,0.300,-0.300


In [5]:

outliers = ['HD66141', 'KPS-1', 'WASP-92','Kepler-41', 'CoRoT-4', 'HD133131B', 'HATS-3','WASP-87A', 'Kepler-40', 'Kepler-435']
 

In [14]:
#appending 'HATS-10' and 'HATS-56' to outliers because they have caused problems
if 'HATS-10' not in outliers:
    outliers.append('HATS-10')
if 'HATS-56' not in outliers:
    outliers.append('HATS-56')
print(len(outliers))

12


# HD66141

## Turbospectrum 1

In [15]:
line_list = "1"
code = "turbospectrum"
star_name = 'HD66141'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

[2023-07-27 17:58:24,534] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 17:58:24,543] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 17:58:24,546] [INFO] [spectrum:estimate_snr:451]: 0.00%


28


[2023-07-27 17:58:25,034] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 17:58:25,538] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 17:58:26,038] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 17:58:26,547] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 17:58:27,075] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 17:58:27,592] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 17:58:28,114] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 17:58:28,633] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 17:58:29,151] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 17:58:29,671] [INFO] [spectrum:estimate_snr:458]: SNR = 174.80
[2023-07-27 17:58:29,764] [INFO] [continuum:__fit_continuum:637]: Resampling spectrum to wave_step: 0.00100 nm (R ~ 472978)
[2023-07-27 17:58:32,182] [INFO] [continuum:__median_filter:307]: Median filter with step: 51
[2023-07-27 17:58:32,298] [INFO] [continuum:__max_filter:298]: Max filter with step: 4

Generating: 4357 1.99 -0.33 0.13 1.53 4.61 2.00 0.60 42310 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  13104.29121  DOF =  8648
   teff = 4357  
   logg = 1.99  
   MH = -0.33  
   alpha = 0.1391  
   Vmic = 1.53  
   Vmac = 4.61  
   Vsini = 2  
   limb_darkening_coeff = 0.6  
   R = 42310  
Cache: 4357 1.99 -0.33 0.13 1.53 4.61 4.00 0.60 42310 loggf [] abund [] vrad []
Cache: 4357 1.99 -0.33 0.13 1.53 4.61 0.00 0.60 42310 loggf [] abund [] vrad []
Cache: 4357 1.99 -0.33 0.13 1.53 4.61 0.00 0.60 42310 loggf [] abund [] vrad []
Cache: 4357 1.99 -0.33 0.13 1.53 4.61 0.00 0.60 42310 loggf [] abund [] vrad []


[2023-07-27 18:01:38,580] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 18:01:38,581] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    1.06	  0.0482	    0.05	  0.0022	  0.0462
Line        487.21	  487.20	  487.26	   11.12	  0.1853	    0.32	  0.0054	  0.0727
Line        489.15	  489.12	  489.21	   29.04	  0.3157	    0.49	  0.0053	  0.0724
Line        489.38	  489.36	  489.40	    3.37	  0.0936	    0.08	  0.0023	  0.0475
Line        490.33	  490.28	  490.37	    4.99	  0.0616	    0.06	  0.0008	  0.0277
Line        492.05	  492.00	  492.09	   11.35	  0.1275	    0.09	  0.0010	  0.0310
Line        492.39	  492.34	  492.42	   11.57	  0.1522	    0.14	  0.0019	  0.0431
Line        492.48	  492.45	  492.51	   25.10	  0.4115	    0.19	  0.0032	  0.0559
Line        493.88	  493.86	  493.91	   12.35	  0.2573	    0.10	  0.0021	  0.0453
Line        493.97	  493.94	  493.99	  112.96	  2.5102	    0.37	  0.0081	  0.0891
Line        496.26	  496.24	  496.28	    5.45	  0.1434	    0.06	  0.0015	  0.0378
Line        

/tmp/ipykernel_1726481/4194638028.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  per_un = (err_vsini/vsini) * 100


110

## Synthe 1

In [25]:
line_list = "1"
code = "synthe"
star_name = 'HD66141'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

[2023-07-27 18:50:12,246] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 18:50:12,251] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 18:50:12,253] [INFO] [spectrum:estimate_snr:451]: 0.00%


28


[2023-07-27 18:50:12,816] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 18:50:13,361] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 18:50:13,894] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 18:50:14,454] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 18:50:14,989] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 18:50:15,508] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 18:50:16,012] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 18:50:16,508] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 18:50:17,023] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 18:50:17,577] [INFO] [spectrum:estimate_snr:458]: SNR = 174.80
[2023-07-27 18:50:17,663] [INFO] [continuum:__fit_continuum:637]: Resampling spectrum to wave_step: 0.00100 nm (R ~ 472978)
[2023-07-27 18:50:20,158] [INFO] [continuum:__median_filter:307]: Median filter with step: 51
[2023-07-27 18:50:20,268] [INFO] [continuum:__max_filter:298]: Max filter with step: 4

Generating: 4357 1.99 -0.33 0.13 1.53 4.61 2.00 0.60 42310 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  13194.30504  DOF =  8648
   teff = 4357  
   logg = 1.99  
   MH = -0.33  
   alpha = 0.1391  
   Vmic = 1.53  
   Vmac = 4.61  
   Vsini = 2  
   limb_darkening_coeff = 0.6  
   R = 42310  
Cache: 4357 1.99 -0.33 0.13 1.53 4.61 4.00 0.60 42310 loggf [] abund [] vrad []
Cache: 4357 1.99 -0.33 0.13 1.53 4.61 0.00 0.60 42310 loggf [] abund [] vrad []
Cache: 4357 1.99 -0.33 0.13 1.53 4.61 0.00 0.60 42310 loggf [] abund [] vrad []
Cache: 4357 1.99 -0.33 0.13 1.53 4.61 0.00 0.60 42310 loggf [] abund [] vrad []


[2023-07-27 18:53:21,762] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 18:53:21,763] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    0.83	  0.0375	    0.04	  0.0017	  0.0404
Line        487.21	  487.20	  487.26	   11.16	  0.1860	    0.33	  0.0055	  0.0736
Line        489.15	  489.12	  489.21	   28.20	  0.3065	    0.48	  0.0052	  0.0715
Line        489.38	  489.36	  489.40	    3.23	  0.0896	    0.08	  0.0022	  0.0465
Line        490.33	  490.28	  490.37	    4.80	  0.0593	    0.07	  0.0009	  0.0296
Line        492.05	  492.00	  492.09	    9.55	  0.1073	    0.07	  0.0008	  0.0288
Line        492.39	  492.34	  492.42	   14.46	  0.1902	    0.17	  0.0023	  0.0473
Line        492.48	  492.45	  492.51	   21.16	  0.3469	    0.17	  0.0028	  0.0522
Line        493.88	  493.86	  493.91	   17.52	  0.3651	    0.14	  0.0030	  0.0542
Line        493.97	  493.94	  493.99	  129.06	  2.8681	    0.43	  0.0096	  0.0969
Line        496.26	  496.24	  496.28	    4.45	  0.1170	    0.05	  0.0012	  0.0343
Line        

/tmp/ipykernel_1726481/738650862.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  per_un = (err_vsini/vsini) * 100


103

## Moog 1

In [34]:
line_list = "1"
code = "moog"
star_name = 'HD66141'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

28


[2023-07-27 19:22:17,519] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 19:22:17,527] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 19:22:17,530] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 19:22:18,423] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 19:22:19,478] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 19:22:20,219] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 19:22:21,242] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 19:22:22,312] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 19:22:23,258] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 19:22:24,287] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 19:22:25,223] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 19:22:26,244] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 19:22:27,105] [INFO] [spectrum:estimate_snr:458

Generating: 4357 1.99 -0.33 0.13 1.53 4.61 2.00 0.60 42310 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  13231.29411  DOF =  8648
   teff = 4357  
   logg = 1.99  
   MH = -0.33  
   alpha = 0.1391  
   Vmic = 1.53  
   Vmac = 4.61  
   Vsini = 2  
   limb_darkening_coeff = 0.6  
   R = 42310  
Cache: 4357 1.99 -0.33 0.13 1.53 4.61 4.00 0.60 42310 loggf [] abund [] vrad []
Cache: 4357 1.99 -0.33 0.13 1.53 4.61 0.00 0.60 42310 loggf [] abund [] vrad []
Cache: 4357 1.99 -0.33 0.13 1.53 4.61 0.00 0.60 42310 loggf [] abund [] vrad []
Cache: 4357 1.99 -0.33 0.13 1.53 4.61 0.00 0.60 42310 loggf [] abund [] vrad []


[2023-07-27 19:23:17,744] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 19:23:17,748] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    1.15	  0.0521	    0.05	  0.0024	  0.0482
Line        487.21	  487.20	  487.26	    4.78	  0.0797	    0.12	  0.0020	  0.0445
Line        489.15	  489.12	  489.21	   19.92	  0.2165	    0.38	  0.0041	  0.0636
Line        489.38	  489.36	  489.40	    3.62	  0.1005	    0.09	  0.0025	  0.0492
Line        490.33	  490.28	  490.37	    5.57	  0.0688	    0.05	  0.0006	  0.0243
Line        492.05	  492.00	  492.09	   22.50	  0.2528	    0.18	  0.0020	  0.0446
Line        492.39	  492.34	  492.42	   24.51	  0.3225	    0.31	  0.0041	  0.0639
Line        492.48	  492.45	  492.51	   31.02	  0.5086	    0.24	  0.0040	  0.0624
Line        493.88	  493.86	  493.91	   14.22	  0.2963	    0.10	  0.0021	  0.0450
Line        493.97	  493.94	  493.99	  107.14	  2.3808	    0.37	  0.0083	  0.0900
Line        496.26	  496.24	  496.28	    5.52	  0.1453	    0.06	  0.0015	  0.0376
Line        

/tmp/ipykernel_1726481/3387682266.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  per_un = (err_vsini/vsini) * 100


101

# WASP-92

## Turbospectrum 1

In [16]:
line_list = "1"
code = "turbospectrum"
star_name = 'WASP-92'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

94


[2023-07-27 18:01:38,795] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 18:01:38,801] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 18:01:38,803] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 18:01:39,365] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 18:01:39,926] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 18:01:40,514] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 18:01:41,075] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 18:01:41,628] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 18:01:42,204] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 18:01:42,833] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 18:01:43,442] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 18:01:44,063] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 18:01:44,668] [INFO] [spectrum:estimate_snr:458

Generating: 6039 4.21 0.10 0.00 0.95 4.42 2.00 0.59 40000 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  137.9240898  DOF =  4460
   teff = 6039  
   logg = 4.21  
   MH = 0.1  
   alpha = 0.023  
   Vmic = 0.95  
   Vmac = 4.42  
   Vsini = 2  
   limb_darkening_coeff = 0.5874886773  
   R = 40000  
Cache: 6039 4.21 0.10 0.00 0.95 4.42 4.00 0.59 40000 loggf [] abund [] vrad []
Cache: 6039 4.21 0.10 0.00 0.95 4.42 24.56 0.59 40000 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  130.2626789  DOF =  4460
   teff = 6039  
   logg = 4.21  
   MH = 0.1  
   alpha = 0.023  
   Vmic = 0.95  
   Vmac = 4.42  
   Vsini = 24.55933729  
   limb_darkening_coeff = 0.5874886773  
   R = 40000  
Cache: 6039 4.21 0.10 0.00 0.95 4.42 26.56 0.59 40000 loggf [] abund [] vrad []
Cache: 6039 4.21 0.10 0.00 0.95 4.42 0.16 0.59 40000 loggf [] abund [] vrad []
Cache: 6039 4.21 0.10 0.00 0.95 4.42 9.85 0.59 40000 loggf [] abund [] vrad []
Iter       3    CHI-SQUARE =  129.4682234  DOF =  4460


[2023-07-27 18:06:45,606] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 18:06:45,607] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    0.42	  0.0353	    3.11	  0.2596	  0.4895
Line        487.21	  487.20	  487.26	    1.12	  0.0295	    7.32	  0.1926	  0.4332
Line        489.15	  489.12	  489.21	    1.49	  0.0262	    7.65	  0.1342	  0.3631
Line        489.38	  489.36	  489.40	    0.83	  0.0414	    2.72	  0.1362	  0.3601
Line        490.33	  490.28	  490.37	    1.14	  0.0292	    4.72	  0.1209	  0.3434
Line        492.05	  492.00	  492.09	    1.09	  0.0280	    8.82	  0.2262	  0.4696
Line        492.39	  492.34	  492.42	    1.17	  0.0254	    5.51	  0.1197	  0.3423
Line        492.48	  492.45	  492.51	    1.33	  0.0381	    7.60	  0.2172	  0.4595
Line        493.88	  493.86	  493.91	    0.80	  0.0298	    5.03	  0.1863	  0.4238
Line        493.97	  493.94	  493.99	    0.77	  0.0285	    3.43	  0.1271	  0.3501
Line        496.26	  496.24	  496.28	    1.03	  0.0545	    9.03	  0.4753	  0.6719
Line        

162

## Synthe 1

In [26]:
line_list = "1"
code = "synthe"
star_name = 'WASP-92'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

94


[2023-07-27 18:53:21,977] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 18:53:21,985] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 18:53:21,988] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 18:53:22,692] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 18:53:23,355] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 18:53:24,080] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 18:53:24,804] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 18:53:25,426] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 18:53:26,008] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 18:53:26,613] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 18:53:27,270] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 18:53:27,992] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 18:53:28,683] [INFO] [spectrum:estimate_snr:458

Generating: 6039 4.21 0.10 0.00 0.95 4.42 2.00 0.59 40000 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  136.9844086  DOF =  4377
   teff = 6039  
   logg = 4.21  
   MH = 0.1  
   alpha = 0.023  
   Vmic = 0.95  
   Vmac = 4.42  
   Vsini = 2  
   limb_darkening_coeff = 0.5874886773  
   R = 40000  
Cache: 6039 4.21 0.10 0.00 0.95 4.42 4.00 0.59 40000 loggf [] abund [] vrad []
Cache: 6039 4.21 0.10 0.00 0.95 4.42 23.34 0.59 40000 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  130.0495165  DOF =  4377
   teff = 6039  
   logg = 4.21  
   MH = 0.1  
   alpha = 0.023  
   Vmic = 0.95  
   Vmac = 4.42  
   Vsini = 23.34027343  
   limb_darkening_coeff = 0.5874886773  
   R = 40000  
Cache: 6039 4.21 0.10 0.00 0.95 4.42 25.34 0.59 40000 loggf [] abund [] vrad []
Cache: 6039 4.21 0.10 0.00 0.95 4.42 0.02 0.59 40000 loggf [] abund [] vrad []
Cache: 6039 4.21 0.10 0.00 0.95 4.42 8.88 0.59 40000 loggf [] abund [] vrad []
Iter       3    CHI-SQUARE =  129.8652821  DOF =  4377


[2023-07-27 18:58:59,536] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 18:58:59,537] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    0.33	  0.0418	    0.94	  0.1178	  0.3237
Line        487.21	  487.20	  487.26	    0.47	  0.0129	    1.81	  0.0502	  0.2209
Line        489.15	  489.12	  489.21	    1.60	  0.0340	   10.63	  0.2263	  0.4707
Line        489.38	  489.36	  489.40	    1.10	  0.0480	    3.79	  0.1649	  0.3975
Line        490.33	  490.28	  490.37	    1.40	  0.0292	    8.10	  0.1687	  0.4065
Line        492.05	  492.00	  492.09	    1.07	  0.0232	    8.47	  0.1841	  0.4244
Line        492.39	  492.34	  492.42	    0.81	  0.0246	    6.04	  0.1831	  0.4215
Line        492.48	  492.45	  492.51	    0.76	  0.0212	    3.24	  0.0900	  0.2959
Line        493.88	  493.86	  493.91	    0.58	  0.0206	    2.87	  0.1025	  0.3145
Line        493.97	  493.94	  493.99	    0.41	  0.0242	    1.41	  0.0831	  0.2801
Line        496.26	  496.24	  496.28	    0.75	  0.0358	    3.75	  0.1786	  0.4129
Line        

154

## Moog 1

In [35]:
line_list = "1"
code = "moog"
star_name = 'WASP-92'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
    

94


[2023-07-27 19:34:47,309] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 19:34:47,316] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 19:34:47,323] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 19:34:48,176] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 19:34:49,016] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 19:34:49,906] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 19:34:50,727] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 19:34:51,605] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 19:34:52,661] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 19:34:53,714] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 19:34:54,615] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 19:34:55,419] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 19:34:56,206] [INFO] [spectrum:estimate_snr:458

Generating: 6039 4.21 0.10 0.00 0.95 4.42 2.00 0.59 40000 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  137.7284589  DOF =  4445
   teff = 6039  
   logg = 4.21  
   MH = 0.1  
   alpha = 0.023  
   Vmic = 0.95  
   Vmac = 4.42  
   Vsini = 2  
   limb_darkening_coeff = 0.5874886773  
   R = 40000  
Cache: 6039 4.21 0.10 0.00 0.95 4.42 4.00 0.59 40000 loggf [] abund [] vrad []
Cache: 6039 4.21 0.10 0.00 0.95 4.42 25.17 0.59 40000 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  128.9066102  DOF =  4445
   teff = 6039  
   logg = 4.21  
   MH = 0.1  
   alpha = 0.023  
   Vmic = 0.95  
   Vmac = 4.42  
   Vsini = 25.17137292  
   limb_darkening_coeff = 0.5874886773  
   R = 40000  
Cache: 6039 4.21 0.10 0.00 0.95 4.42 27.17 0.59 40000 loggf [] abund [] vrad []
Cache: 6039 4.21 0.10 0.00 0.95 4.42 0.00 0.59 40000 loggf [] abund [] vrad []
Cache: 6039 4.21 0.10 0.00 0.95 4.42 10.92 0.59 40000 loggf [] abund [] vrad []
Iter       3    CHI-SQUARE =  127.8210706  DOF =  4445

[2023-07-27 19:39:00,812] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 19:39:00,813] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    0.44	  0.0261	    2.91	  0.1713	  0.4022
Line        487.21	  487.20	  487.26	    0.98	  0.0281	    4.97	  0.1421	  0.3717
Line        489.15	  489.12	  489.21	    2.22	  0.0364	   14.88	  0.2439	  0.4899
Line        489.38	  489.36	  489.40	    1.07	  0.0487	    5.56	  0.2528	  0.4917
Line        490.33	  490.28	  490.37	    1.32	  0.0288	    4.69	  0.1019	  0.3159
Line        492.05	  492.00	  492.09	    1.01	  0.0197	    5.24	  0.1028	  0.3176
Line        492.39	  492.34	  492.42	    1.05	  0.0239	    6.34	  0.1441	  0.3753
Line        492.48	  492.45	  492.51	    0.85	  0.0266	    4.25	  0.1329	  0.3589
Line        493.88	  493.86	  493.91	    0.86	  0.0295	    4.33	  0.1492	  0.3798
Line        493.97	  493.94	  493.99	    0.81	  0.0336	    4.94	  0.2060	  0.4447
Line        496.26	  496.24	  496.28	    0.59	  0.0330	    2.34	  0.1299	  0.3508
Line        

152

# Kepler-41

## Turbospectrum 1

In [17]:
line_list = "1"
code = "turbospectrum"
star_name = 'Kepler-41'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

78


[2023-07-27 18:06:45,804] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 18:06:45,808] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 18:06:45,810] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 18:06:46,361] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 18:06:46,922] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 18:06:47,478] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 18:06:48,038] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 18:06:48,591] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 18:06:49,159] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 18:06:49,721] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 18:06:50,271] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 18:06:50,832] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 18:06:51,396] [INFO] [spectrum:estimate_snr:458

Generating: 5766 4.32 0.21 0.00 0.47 3.42 2.00 0.62 40000 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  6.089187463  DOF =  789
   teff = 5766  
   logg = 4.32  
   MH = 0.21  
   alpha = -0.0067  
   Vmic = 0.47  
   Vmac = 3.42  
   Vsini = 2  
   limb_darkening_coeff = 0.6210051656  
   R = 40000  
Cache: 5766 4.32 0.21 0.00 0.47 3.42 4.00 0.62 40000 loggf [] abund [] vrad []
Cache: 5766 4.32 0.21 0.00 0.47 3.42 26.29 0.62 40000 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  4.523400381  DOF =  789
   teff = 5766  
   logg = 4.32  
   MH = 0.21  
   alpha = -0.0067  
   Vmic = 0.47  
   Vmac = 3.42  
   Vsini = 26.29147946  
   limb_darkening_coeff = 0.6210051656  
   R = 40000  
Cache: 5766 4.32 0.21 0.00 0.47 3.42 28.29 0.62 40000 loggf [] abund [] vrad []
Cache: 5766 4.32 0.21 0.00 0.47 3.42 35.25 0.62 40000 loggf [] abund [] vrad []
Iter       3    CHI-SQUARE =  4.480346977  DOF =  789
   teff = 5766  
   logg = 4.32  
   MH = 0.21  
   alpha = -0.0067  
   Vm

[2023-07-27 18:10:34,618] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 18:10:34,619] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    0.00	  0.0039	    0.05	  0.0504	  0.1588
Line        487.21	  487.20	  487.26	    0.87	  0.1736	    1.28	  0.2560	  0.4619
Line        489.15	  489.12	  489.21	    0.74	  0.0495	    1.92	  0.1282	  0.3467
Line        489.38	  489.36	  489.40	    0.58	  0.1921	    1.70	  0.5678	  0.6526
Line        490.33	  490.28	  490.37	    0.15	  0.0170	    0.99	  0.1097	  0.3142
Line        492.05	  492.00	  492.09	    0.10	  0.0110	    2.74	  0.3045	  0.5235
Line        492.39	  492.34	  492.42	    0.08	  0.0094	    4.06	  0.4514	  0.6374
Line        492.48	  492.45	  492.51	    0.03	  0.0048	    0.97	  0.1614	  0.3720
Line        493.88	  493.86	  493.91	    0.04	  0.0060	    1.14	  0.1627	  0.3773
Line        493.97	  493.94	  493.99	    0.01	  0.0037	    0.30	  0.0995	  0.2732
Line        496.26	  496.24	  496.28	-9999.00	-9999.0000	-9999.00	-9999.0000	-9999.0000
Line  

162

## Synthe 1

In [27]:
line_list = "1"
code = "synthe"
star_name = 'Kepler-41'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

78


[2023-07-27 18:58:59,706] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 18:58:59,709] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 18:58:59,712] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 18:59:00,244] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 18:59:00,786] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 18:59:01,387] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 18:59:01,966] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 18:59:02,572] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 18:59:03,140] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 18:59:03,726] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 18:59:04,313] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 18:59:04,883] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 18:59:05,475] [INFO] [spectrum:estimate_snr:458

Generating: 5766 4.32 0.21 0.00 0.47 3.42 2.00 0.62 40000 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  2.682851379  DOF =  667
   teff = 5766  
   logg = 4.32  
   MH = 0.21  
   alpha = -0.0067  
   Vmic = 0.47  
   Vmac = 3.42  
   Vsini = 2  
   limb_darkening_coeff = 0.6210051656  
   R = 40000  
Cache: 5766 4.32 0.21 0.00 0.47 3.42 4.00 0.62 40000 loggf [] abund [] vrad []
Cache: 5766 4.32 0.21 0.00 0.47 3.42 18.57 0.62 40000 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  2.142203473  DOF =  667
   teff = 5766  
   logg = 4.32  
   MH = 0.21  
   alpha = -0.0067  
   Vmic = 0.47  
   Vmac = 3.42  
   Vsini = 18.5678338  
   limb_darkening_coeff = 0.6210051656  
   R = 40000  
Cache: 5766 4.32 0.21 0.00 0.47 3.42 20.57 0.62 40000 loggf [] abund [] vrad []
Cache: 5766 4.32 0.21 0.00 0.47 3.42 30.86 0.62 40000 loggf [] abund [] vrad []
Iter       3    CHI-SQUARE =  1.933224966  DOF =  667
   teff = 5766  
   logg = 4.32  
   MH = 0.21  
   alpha = -0.0067  
   Vmi

[2023-07-27 19:02:44,152] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 19:02:44,153] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    0.00	  0.0001	    0.00	  0.0015	  0.0315
Line        487.21	  487.20	  487.26	    0.00	  0.0006	    0.02	  0.0112	  0.0864
Line        489.15	  489.12	  489.21	    0.01	  0.0007	    0.06	  0.0087	  0.0871
Line        489.38	  489.36	  489.40	    0.04	  0.0089	    0.59	  0.1179	  0.3135
Line        490.33	  490.28	  490.37	    0.04	  0.0061	    0.48	  0.0691	  0.2459
Line        492.05	  492.00	  492.09	    0.03	  0.0036	    0.76	  0.1089	  0.3087
Line        492.39	  492.34	  492.42	    0.02	  0.0047	    0.60	  0.1505	  0.3470
Line        492.48	  492.45	  492.51	    0.00	  0.0000	    0.00	  0.0002	  0.0122
Line        493.88	  493.86	  493.91	    0.01	  0.0011	    0.21	  0.0233	  0.1448
Line        493.97	  493.94	  493.99	    0.03	  0.0099	    1.20	  0.3989	  0.5470
Line        496.26	  496.24	  496.28	    0.00	  0.0001	    0.01	  0.0029	  0.0493
Line        

157

## Moog 1

In [36]:
line_list = "1"
code = "moog"
star_name = 'Kepler-41'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
    

78


[2023-07-27 19:39:01,090] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 19:39:01,095] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 19:39:01,098] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 19:39:01,756] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 19:39:02,392] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 19:39:03,003] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 19:39:03,695] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 19:39:04,332] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 19:39:05,029] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 19:39:05,726] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 19:39:06,411] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 19:39:07,065] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 19:39:07,730] [INFO] [spectrum:estimate_snr:458

Generating: 5766 4.32 0.21 0.00 0.47 3.42 2.00 0.62 40000 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  2.551517471  DOF =  689
   teff = 5766  
   logg = 4.32  
   MH = 0.21  
   alpha = -0.0067  
   Vmic = 0.47  
   Vmac = 3.42  
   Vsini = 2  
   limb_darkening_coeff = 0.6210051656  
   R = 40000  
Cache: 5766 4.32 0.21 0.00 0.47 3.42 4.00 0.62 40000 loggf [] abund [] vrad []
Cache: 5766 4.32 0.21 0.00 0.47 3.42 21.10 0.62 40000 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  1.865098753  DOF =  689
   teff = 5766  
   logg = 4.32  
   MH = 0.21  
   alpha = -0.0067  
   Vmic = 0.47  
   Vmac = 3.42  
   Vsini = 21.10350845  
   limb_darkening_coeff = 0.6210051656  
   R = 40000  
Cache: 5766 4.32 0.21 0.00 0.47 3.42 23.10 0.62 40000 loggf [] abund [] vrad []
Cache: 5766 4.32 0.21 0.00 0.47 3.42 36.92 0.62 40000 loggf [] abund [] vrad []
Iter       3    CHI-SQUARE =  1.611817589  DOF =  689
   teff = 5766  
   logg = 4.32  
   MH = 0.21  
   alpha = -0.0067  
   Vm

[2023-07-27 19:41:03,564] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 19:41:03,567] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    0.01	  0.0031	    0.12	  0.0386	  0.1701
Line        487.21	  487.20	  487.26	    0.01	  0.0015	    0.21	  0.0417	  0.1865
Line        489.15	  489.12	  489.21	    0.02	  0.0022	    0.80	  0.0890	  0.2830
Line        489.38	  489.36	  489.40	    0.00	  0.0001	    0.01	  0.0026	  0.0413
Line        490.33	  490.28	  490.37	    0.00	  0.0004	    0.03	  0.0081	  0.0807
Line        492.05	  492.00	  492.09	    0.00	  0.0005	    0.06	  0.0103	  0.0942
Line        492.39	  492.34	  492.42	    0.02	  0.0024	    0.84	  0.0931	  0.2894
Line        492.48	  492.45	  492.51	    0.00	  0.0000	    0.00	  0.0003	  0.0140
Line        493.88	  493.86	  493.91	    0.00	  0.0004	    0.04	  0.0091	  0.0852
Line        493.97	  493.94	  493.99	    0.00	  0.0004	    0.02	  0.0083	  0.0791
Line        496.26	  496.24	  496.28	    0.00	  0.0003	    0.04	  0.0056	  0.0702
Line        

153

# CoRoT-4

## Turbospectrum 1

In [18]:
line_list = "1"
code = "turbospectrum"
star_name = 'CoRoT-4'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

59


[2023-07-27 18:10:34,783] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 18:10:34,787] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 18:10:34,791] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 18:10:35,451] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 18:10:36,264] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 18:10:37,007] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 18:10:37,705] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 18:10:38,325] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 18:10:38,925] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 18:10:39,614] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 18:10:40,402] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 18:10:41,553] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 18:10:42,375] [INFO] [spectrum:estimate_snr:458

Generating: 6238 4.55 0.18 0.00 1.23 4.49 2.00 0.58 56990 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  112740.9619  DOF =  8654
   teff = 6238  
   logg = 4.55  
   MH = 0.18  
   alpha = 0.0014  
   Vmic = 1.23  
   Vmac = 4.49  
   Vsini = 2  
   limb_darkening_coeff = 0.5778855778  
   R = 56990  
Cache: 6238 4.55 0.18 0.00 1.23 4.49 4.00 0.58 56990 loggf [] abund [] vrad []
Cache: 6238 4.55 0.18 0.00 1.23 4.49 21.27 0.58 56990 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  48461.98098  DOF =  8654
   teff = 6238  
   logg = 4.55  
   MH = 0.18  
   alpha = 0.0014  
   Vmic = 1.23  
   Vmac = 4.49  
   Vsini = 21.27211776  
   limb_darkening_coeff = 0.5778855778  
   R = 56990  
Cache: 6238 4.55 0.18 0.00 1.23 4.49 23.27 0.58 56990 loggf [] abund [] vrad []
Cache: 6238 4.55 0.18 0.00 1.23 4.49 37.56 0.58 56990 loggf [] abund [] vrad []
Iter       3    CHI-SQUARE =  33166.64617  DOF =  8654
   teff = 6238  
   logg = 4.55  
   MH = 0.18  
   alpha = 0.0014  
   Vm

[2023-07-27 18:13:39,073] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 18:13:39,074] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	   76.70	  3.4862	    4.35	  0.1976	  0.4347
Line        487.21	  487.20	  487.26	   62.30	  1.0560	    2.97	  0.0503	  0.2224
Line        489.15	  489.12	  489.21	   62.99	  0.6773	    1.88	  0.0202	  0.1414
Line        489.38	  489.36	  489.40	    0.73	  0.0204	    0.02	  0.0006	  0.0240
Line        490.33	  490.28	  490.37	   13.57	  0.1655	    0.32	  0.0039	  0.0624
Line        492.05	  492.00	  492.09	  106.97	  1.2156	    1.68	  0.0191	  0.1373
Line        492.39	  492.34	  492.42	   42.46	  0.5587	    0.68	  0.0089	  0.0937
Line        492.48	  492.45	  492.51	   23.63	  0.3873	    0.39	  0.0064	  0.0791
Line        493.88	  493.86	  493.91	   56.44	  1.1759	    0.78	  0.0163	  0.1263
Line        493.97	  493.94	  493.99	   37.51	  0.8337	    0.52	  0.0115	  0.1059
Line        496.26	  496.24	  496.28	   10.27	  0.2777	    0.09	  0.0025	  0.0491
Line        

162

## Synthe 1

In [28]:
line_list = "1"
code = "synthe"
star_name = 'CoRoT-4'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

59


[2023-07-27 19:02:44,289] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 19:02:44,296] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 19:02:44,300] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 19:02:44,835] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 19:02:45,413] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 19:02:46,011] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 19:02:46,581] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 19:02:47,140] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 19:02:47,655] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 19:02:48,212] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 19:02:48,765] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 19:02:49,316] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 19:02:49,896] [INFO] [spectrum:estimate_snr:458

Generating: 6238 4.55 0.18 0.00 1.23 4.49 2.00 0.58 56990 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  122393.3749  DOF =  8647
   teff = 6238  
   logg = 4.55  
   MH = 0.18  
   alpha = 0.0014  
   Vmic = 1.23  
   Vmac = 4.49  
   Vsini = 2  
   limb_darkening_coeff = 0.5778855778  
   R = 56990  
Cache: 6238 4.55 0.18 0.00 1.23 4.49 4.00 0.58 56990 loggf [] abund [] vrad []
Cache: 6238 4.55 0.18 0.00 1.23 4.49 22.29 0.58 56990 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  51658.35933  DOF =  8647
   teff = 6238  
   logg = 4.55  
   MH = 0.18  
   alpha = 0.0014  
   Vmic = 1.23  
   Vmac = 4.49  
   Vsini = 22.28982195  
   limb_darkening_coeff = 0.5778855778  
   R = 56990  
Cache: 6238 4.55 0.18 0.00 1.23 4.49 24.29 0.58 56990 loggf [] abund [] vrad []
Cache: 6238 4.55 0.18 0.00 1.23 4.49 38.63 0.58 56990 loggf [] abund [] vrad []
Iter       3    CHI-SQUARE =  38186.16319  DOF =  8647
   teff = 6238  
   logg = 4.55  
   MH = 0.18  
   alpha = 0.0014  
   Vm

[2023-07-27 19:05:34,170] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 19:05:34,171] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    1.24	  0.0563	    0.13	  0.0059	  0.0754
Line        487.21	  487.20	  487.26	   37.07	  0.6283	    1.10	  0.0187	  0.1355
Line        489.15	  489.12	  489.21	  314.86	  3.3856	    4.63	  0.0498	  0.2220
Line        489.38	  489.36	  489.40	    0.83	  0.0231	    0.03	  0.0007	  0.0264
Line        490.33	  490.28	  490.37	    4.93	  0.0608	    0.13	  0.0015	  0.0391
Line        492.05	  492.00	  492.09	  521.01	  5.8541	    4.03	  0.0453	  0.2116
Line        492.39	  492.34	  492.42	   98.80	  1.2999	    1.24	  0.0163	  0.1267
Line        492.48	  492.45	  492.51	    6.33	  0.1039	    0.10	  0.0017	  0.0405
Line        493.88	  493.86	  493.91	   68.31	  1.4232	    0.71	  0.0148	  0.1203
Line        493.97	  493.94	  493.99	   87.42	  1.9426	    0.88	  0.0196	  0.1383
Line        496.26	  496.24	  496.28	    2.64	  0.0695	    0.03	  0.0007	  0.0267
Line        

157

## Moog 1

In [37]:
line_list = "1"
code = "moog"
star_name = 'CoRoT-4'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

59


[2023-07-27 19:41:03,749] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 19:41:03,754] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 19:41:03,756] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 19:41:04,355] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 19:41:04,978] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 19:41:05,647] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 19:41:06,244] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 19:41:06,828] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 19:41:07,392] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 19:41:08,016] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 19:41:08,660] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 19:41:09,336] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 19:41:10,054] [INFO] [spectrum:estimate_snr:458

Generating: 6238 4.55 0.18 0.00 1.23 4.49 2.00 0.58 56990 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  10779.50423  DOF =  8645
   teff = 6238  
   logg = 4.55  
   MH = 0.18  
   alpha = 0.0014  
   Vmic = 1.23  
   Vmac = 4.49  
   Vsini = 2  
   limb_darkening_coeff = 0.5778855778  
   R = 56990  
Cache: 6238 4.55 0.18 0.00 1.23 4.49 4.00 0.58 56990 loggf [] abund [] vrad []
Cache: 6238 4.55 0.18 0.00 1.23 4.49 8.43 0.58 56990 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  5703.853081  DOF =  8645
   teff = 6238  
   logg = 4.55  
   MH = 0.18  
   alpha = 0.0014  
   Vmic = 1.23  
   Vmac = 4.49  
   Vsini = 8.4279158  
   limb_darkening_coeff = 0.5778855778  
   R = 56990  
Cache: 6238 4.55 0.18 0.00 1.23 4.49 10.43 0.58 56990 loggf [] abund [] vrad []
Cache: 6238 4.55 0.18 0.00 1.23 4.49 7.26 0.58 56990 loggf [] abund [] vrad []
Iter       3    CHI-SQUARE =  5216.158664  DOF =  8645
   teff = 6238  
   logg = 4.55  
   MH = 0.18  
   alpha = 0.0014  
   Vmic =

[2023-07-27 19:42:02,044] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 19:42:02,046] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    3.57	  0.1621	    0.31	  0.0141	  0.1160
Line        487.21	  487.20	  487.26	    6.84	  0.1160	    0.22	  0.0037	  0.0600
Line        489.15	  489.12	  489.21	   15.55	  0.1690	    0.33	  0.0036	  0.0596
Line        489.38	  489.36	  489.40	    1.06	  0.0294	    0.03	  0.0008	  0.0284
Line        490.33	  490.28	  490.37	    9.91	  0.1209	    0.18	  0.0021	  0.0461
Line        492.05	  492.00	  492.09	   20.13	  0.2287	    0.18	  0.0020	  0.0450
Line        492.39	  492.34	  492.42	   30.05	  0.3954	    0.30	  0.0040	  0.0626
Line        492.48	  492.45	  492.51	   11.60	  0.1901	    0.15	  0.0024	  0.0489
Line        493.88	  493.86	  493.91	   14.54	  0.3030	    0.16	  0.0033	  0.0568
Line        493.97	  493.94	  493.99	   10.32	  0.2293	    0.11	  0.0025	  0.0491
Line        496.26	  496.24	  496.28	    4.39	  0.1156	    0.04	  0.0011	  0.0321
Line        

153

# HD133131B

## Turbospectrum 1

In [19]:
line_list = "1"
code = "turbospectrum"
star_name = 'HD133131B'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

41


[2023-07-27 18:13:39,294] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 18:13:39,302] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 18:13:39,306] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 18:13:39,939] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 18:13:40,750] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 18:13:41,452] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 18:13:42,123] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 18:13:42,771] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 18:13:43,472] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 18:13:44,180] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 18:13:44,890] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 18:13:45,559] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 18:13:46,321] [INFO] [spectrum:estimate_snr:458

Generating: 5751 4.37 -0.28 0.11 0.90 3.29 2.00 0.59 74450 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  7216.030916  DOF =  8648
   teff = 5751  
   logg = 4.37  
   MH = -0.28  
   alpha = 0.1256  
   Vmic = 0.9  
   Vmac = 3.29  
   Vsini = 2  
   limb_darkening_coeff = 0.5943380721  
   R = 74450  
Cache: 5751 4.37 -0.28 0.11 0.90 3.29 4.00 0.59 74450 loggf [] abund [] vrad []
Cache: 5751 4.37 -0.28 0.11 0.90 3.29 0.43 0.59 74450 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  6143.201848  DOF =  8648
   teff = 5751  
   logg = 4.37  
   MH = -0.28  
   alpha = 0.1256  
   Vmic = 0.9  
   Vmac = 3.29  
   Vsini = 0.4333460407  
   limb_darkening_coeff = 0.5943380721  
   R = 74450  
Cache: 5751 4.37 -0.28 0.11 0.90 3.29 2.43 0.59 74450 loggf [] abund [] vrad []
Cache: 5751 4.37 -0.28 0.11 0.90 3.29 0.00 0.59 74450 loggf [] abund [] vrad []
Cache: 5751 4.37 -0.28 0.11 0.90 3.29 0.00 0.59 74450 loggf [] abund [] vrad []
Cache: 5751 4.37 -0.28 0.11 0.90 3.29 0.00 0.5

[2023-07-27 18:16:31,514] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 18:16:31,515] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    3.33	  0.1515	    0.33	  0.0151	  0.1202
Line        487.21	  487.20	  487.26	   10.51	  0.1781	    0.60	  0.0101	  0.0996
Line        489.15	  489.12	  489.21	    4.96	  0.0539	    0.13	  0.0014	  0.0371
Line        489.38	  489.36	  489.40	    0.30	  0.0083	    0.01	  0.0004	  0.0185
Line        490.33	  490.28	  490.37	    3.16	  0.0385	    0.07	  0.0009	  0.0292
Line        492.05	  492.00	  492.09	    5.09	  0.0578	    0.05	  0.0005	  0.0231
Line        492.39	  492.34	  492.42	    7.08	  0.0932	    0.11	  0.0014	  0.0372
Line        492.48	  492.45	  492.51	    6.68	  0.1096	    0.11	  0.0018	  0.0416
Line        493.88	  493.86	  493.91	    3.19	  0.0665	    0.04	  0.0008	  0.0275
Line        493.97	  493.94	  493.99	    4.59	  0.1019	    0.05	  0.0010	  0.0317
Line        496.26	  496.24	  496.28	    0.95	  0.0256	    0.01	  0.0003	  0.0172
Line        

/tmp/ipykernel_1726481/3947378323.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  per_un = (err_vsini/vsini) * 100


142

## Synthe 1

In [29]:
line_list = "1"
code = "synthe"
star_name = 'HD133131B'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

41


[2023-07-27 19:05:34,292] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 19:05:34,296] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 19:05:34,298] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 19:05:34,845] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 19:05:35,406] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 19:05:35,966] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 19:05:36,540] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 19:05:37,093] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 19:05:37,676] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 19:05:38,241] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 19:05:38,816] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 19:05:39,394] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 19:05:39,987] [INFO] [spectrum:estimate_snr:458

Generating: 5751 4.37 -0.28 0.11 0.90 3.29 2.00 0.59 74450 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  7603.56095  DOF =  8648
   teff = 5751  
   logg = 4.37  
   MH = -0.28  
   alpha = 0.1256  
   Vmic = 0.9  
   Vmac = 3.29  
   Vsini = 2  
   limb_darkening_coeff = 0.5943380721  
   R = 74450  
Cache: 5751 4.37 -0.28 0.11 0.90 3.29 4.00 0.59 74450 loggf [] abund [] vrad []
Cache: 5751 4.37 -0.28 0.11 0.90 3.29 0.35 0.59 74450 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  6444.682472  DOF =  8648
   teff = 5751  
   logg = 4.37  
   MH = -0.28  
   alpha = 0.1256  
   Vmic = 0.9  
   Vmac = 3.29  
   Vsini = 0.3479607286  
   limb_darkening_coeff = 0.5943380721  
   R = 74450  
Cache: 5751 4.37 -0.28 0.11 0.90 3.29 2.35 0.59 74450 loggf [] abund [] vrad []
Cache: 5751 4.37 -0.28 0.11 0.90 3.29 0.00 0.59 74450 loggf [] abund [] vrad []
Cache: 5751 4.37 -0.28 0.11 0.90 3.29 0.00 0.59 74450 loggf [] abund [] vrad []
Cache: 5751 4.37 -0.28 0.11 0.90 3.29 0.00 0.59

[2023-07-27 19:08:31,912] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 19:08:31,914] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    3.28	  0.1492	    0.33	  0.0150	  0.1197
Line        487.21	  487.20	  487.26	    9.36	  0.1587	    0.53	  0.0091	  0.0944
Line        489.15	  489.12	  489.21	    3.80	  0.0413	    0.10	  0.0011	  0.0328
Line        489.38	  489.36	  489.40	    0.35	  0.0097	    0.01	  0.0004	  0.0199
Line        490.33	  490.28	  490.37	    2.45	  0.0299	    0.06	  0.0007	  0.0272
Line        492.05	  492.00	  492.09	    3.36	  0.0382	    0.03	  0.0004	  0.0190
Line        492.39	  492.34	  492.42	    9.53	  0.1253	    0.14	  0.0018	  0.0424
Line        492.48	  492.45	  492.51	    5.55	  0.0910	    0.09	  0.0015	  0.0381
Line        493.88	  493.86	  493.91	    5.52	  0.1150	    0.06	  0.0013	  0.0355
Line        493.97	  493.94	  493.99	    8.25	  0.1834	    0.09	  0.0019	  0.0434
Line        496.26	  496.24	  496.28	    0.65	  0.0175	    0.01	  0.0002	  0.0143
Line        

/tmp/ipykernel_1726481/891772174.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  per_un = (err_vsini/vsini) * 100


135

## Moog 1

In [38]:
line_list = "1"
code = "moog"
star_name = 'HD133131B'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

41


[2023-07-27 19:42:02,221] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 19:42:02,226] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 19:42:02,228] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 19:42:02,950] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 19:42:03,640] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 19:42:04,339] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 19:42:04,933] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 19:42:05,517] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 19:42:06,179] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 19:42:06,815] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 19:42:07,480] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 19:42:08,206] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 19:42:08,991] [INFO] [spectrum:estimate_snr:458

Generating: 5751 4.37 -0.28 0.11 0.90 3.29 2.00 0.59 74450 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  7314.301358  DOF =  8648
   teff = 5751  
   logg = 4.37  
   MH = -0.28  
   alpha = 0.1256  
   Vmic = 0.9  
   Vmac = 3.29  
   Vsini = 2  
   limb_darkening_coeff = 0.5943380721  
   R = 74450  
Cache: 5751 4.37 -0.28 0.11 0.90 3.29 4.00 0.59 74450 loggf [] abund [] vrad []
Cache: 5751 4.37 -0.28 0.11 0.90 3.29 0.60 0.59 74450 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  6387.944014  DOF =  8648
   teff = 5751  
   logg = 4.37  
   MH = -0.28  
   alpha = 0.1256  
   Vmic = 0.9  
   Vmac = 3.29  
   Vsini = 0.6005292167  
   limb_darkening_coeff = 0.5943380721  
   R = 74450  
Cache: 5751 4.37 -0.28 0.11 0.90 3.29 2.60 0.59 74450 loggf [] abund [] vrad []
Cache: 5751 4.37 -0.28 0.11 0.90 3.29 0.00 0.59 74450 loggf [] abund [] vrad []
Cache: 5751 4.37 -0.28 0.11 0.90 3.29 0.00 0.59 74450 loggf [] abund [] vrad []
Cache: 5751 4.37 -0.28 0.11 0.90 3.29 0.00 0.5

[2023-07-27 19:42:53,796] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 19:42:53,802] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    3.54	  0.1610	    0.35	  0.0161	  0.1241
Line        487.21	  487.20	  487.26	    8.71	  0.1476	    0.47	  0.0080	  0.0889
Line        489.15	  489.12	  489.21	    5.00	  0.0543	    0.13	  0.0014	  0.0375
Line        489.38	  489.36	  489.40	    0.37	  0.0102	    0.02	  0.0004	  0.0204
Line        490.33	  490.28	  490.37	    3.58	  0.0437	    0.08	  0.0010	  0.0312
Line        492.05	  492.00	  492.09	    4.48	  0.0509	    0.04	  0.0005	  0.0223
Line        492.39	  492.34	  492.42	    8.24	  0.1085	    0.13	  0.0017	  0.0410
Line        492.48	  492.45	  492.51	    6.61	  0.1084	    0.11	  0.0018	  0.0415
Line        493.88	  493.86	  493.91	    3.51	  0.0731	    0.04	  0.0009	  0.0290
Line        493.97	  493.94	  493.99	    4.71	  0.1047	    0.05	  0.0011	  0.0324
Line        496.26	  496.24	  496.28	    1.12	  0.0304	    0.01	  0.0004	  0.0186
Line        

/tmp/ipykernel_1726481/2573938347.py:32: RuntimeWarning: divide by zero encountered in double_scalars
  per_un = (err_vsini/vsini) * 100


133

# HATS-3

## Turbospectrum 1

In [20]:
line_list = "1"
code = "turbospectrum"
star_name = 'HATS-3'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

27


[2023-07-27 18:16:31,888] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 18:16:31,895] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 18:16:31,898] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 18:16:32,611] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 18:16:33,433] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 18:16:34,103] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 18:16:34,793] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 18:16:35,455] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 18:16:36,087] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 18:16:36,781] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 18:16:37,512] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 18:16:38,173] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 18:16:38,872] [INFO] [spectrum:estimate_snr:458

Generating: 6347 4.52 -0.02 0.01 1.41 5.03 2.00 0.56 48000 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  33307.74892  DOF =  8652
   teff = 6347  
   logg = 4.52  
   MH = -0.02  
   alpha = 0.0554  
   Vmic = 1.41  
   Vmac = 5.03  
   Vsini = 2  
   limb_darkening_coeff = 0.5559810971  
   R = 48000  
Cache: 6347 4.52 -0.02 0.01 1.41 5.03 4.00 0.56 48000 loggf [] abund [] vrad []
Cache: 6347 4.52 -0.02 0.01 1.41 5.03 24.60 0.56 48000 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  13439.46625  DOF =  8652
   teff = 6347  
   logg = 4.52  
   MH = -0.02  
   alpha = 0.0554  
   Vmic = 1.41  
   Vmac = 5.03  
   Vsini = 24.59828154  
   limb_darkening_coeff = 0.5559810971  
   R = 48000  
Cache: 6347 4.52 -0.02 0.01 1.41 5.03 26.60 0.56 48000 loggf [] abund [] vrad []
Cache: 6347 4.52 -0.02 0.01 1.41 5.03 41.69 0.56 48000 loggf [] abund [] vrad []
Iter       3    CHI-SQUARE =  9846.081542  DOF =  8652
   teff = 6347  
   logg = 4.52  
   MH = -0.02  
   alpha = 0.0554

[2023-07-27 18:19:35,155] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 18:19:35,156] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    0.94	  0.0411	    0.01	  0.0005	  0.0213
Line        487.21	  487.20	  487.26	   83.31	  1.4120	    1.04	  0.0176	  0.1316
Line        489.15	  489.12	  489.21	  194.23	  2.0885	    2.36	  0.0254	  0.1585
Line        489.38	  489.36	  489.40	    0.15	  0.0041	    0.00	  0.0000	  0.0069
Line        490.33	  490.28	  490.37	   35.70	  0.4354	    0.42	  0.0051	  0.0710
Line        492.05	  492.00	  492.09	  485.08	  5.5123	    3.29	  0.0374	  0.1923
Line        492.39	  492.34	  492.42	   60.96	  0.8022	    0.65	  0.0085	  0.0916
Line        492.48	  492.45	  492.51	   30.30	  0.4967	    0.29	  0.0047	  0.0684
Line        493.88	  493.86	  493.91	   50.09	  1.0222	    0.50	  0.0102	  0.0999
Line        493.97	  493.94	  493.99	   19.29	  0.4286	    0.19	  0.0042	  0.0643
Line        496.26	  496.24	  496.28	    4.61	  0.1246	    0.04	  0.0011	  0.0328
Line        

176

## Synthe 1

In [30]:
line_list = "1"
code = "synthe"
star_name = 'HATS-3'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

27


[2023-07-27 19:08:32,211] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 19:08:32,217] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 19:08:32,220] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 19:08:32,818] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 19:08:33,406] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 19:08:34,000] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 19:08:34,593] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 19:08:35,204] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 19:08:35,827] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 19:08:36,442] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 19:08:37,043] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 19:08:37,640] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 19:08:38,250] [INFO] [spectrum:estimate_snr:458

Generating: 6347 4.52 -0.02 0.01 1.41 5.03 2.00 0.56 48000 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  33538.11051  DOF =  8658
   teff = 6347  
   logg = 4.52  
   MH = -0.02  
   alpha = 0.0554  
   Vmic = 1.41  
   Vmac = 5.03  
   Vsini = 2  
   limb_darkening_coeff = 0.5559810971  
   R = 48000  
Cache: 6347 4.52 -0.02 0.01 1.41 5.03 4.00 0.56 48000 loggf [] abund [] vrad []
Cache: 6347 4.52 -0.02 0.01 1.41 5.03 24.56 0.56 48000 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  13565.14748  DOF =  8658
   teff = 6347  
   logg = 4.52  
   MH = -0.02  
   alpha = 0.0554  
   Vmic = 1.41  
   Vmac = 5.03  
   Vsini = 24.55518104  
   limb_darkening_coeff = 0.5559810971  
   R = 48000  
Cache: 6347 4.52 -0.02 0.01 1.41 5.03 26.56 0.56 48000 loggf [] abund [] vrad []
Cache: 6347 4.52 -0.02 0.01 1.41 5.03 41.39 0.56 48000 loggf [] abund [] vrad []
Iter       3    CHI-SQUARE =  10045.82778  DOF =  8658
   teff = 6347  
   logg = 4.52  
   MH = -0.02  
   alpha = 0.0554

[2023-07-27 19:11:43,649] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 19:11:43,653] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    0.58	  0.0262	    0.01	  0.0003	  0.0172
Line        487.21	  487.20	  487.26	   75.38	  1.2563	    0.96	  0.0159	  0.1251
Line        489.15	  489.12	  489.21	  188.96	  2.0318	    2.33	  0.0250	  0.1574
Line        489.38	  489.36	  489.40	    0.09	  0.0025	    0.00	  0.0000	  0.0054
Line        490.33	  490.28	  490.37	   35.65	  0.4348	    0.42	  0.0051	  0.0712
Line        492.05	  492.00	  492.09	  505.75	  5.6826	    3.42	  0.0384	  0.1948
Line        492.39	  492.34	  492.42	   58.92	  0.7653	    0.63	  0.0081	  0.0896
Line        492.48	  492.45	  492.51	   29.84	  0.4891	    0.28	  0.0047	  0.0678
Line        493.88	  493.86	  493.91	   50.43	  1.0507	    0.50	  0.0104	  0.1011
Line        493.97	  493.94	  493.99	   19.96	  0.4339	    0.20	  0.0043	  0.0647
Line        496.26	  496.24	  496.28	    6.11	  0.1608	    0.05	  0.0014	  0.0368
Line        

171

## Moog 1

In [39]:
line_list = "1"
code = "moog"
star_name = 'HATS-3'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
  

27


[2023-07-27 19:42:54,141] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 19:42:54,151] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 19:42:54,154] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 19:42:54,821] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 19:42:55,449] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 19:42:56,161] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 19:42:56,857] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 19:42:57,575] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 19:42:58,263] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 19:42:58,919] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 19:42:59,567] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 19:43:00,266] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 19:43:00,910] [INFO] [spectrum:estimate_snr:458

Generating: 6347 4.52 -0.02 0.01 1.41 5.03 2.00 0.56 48000 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  34800.96714  DOF =  8652
   teff = 6347  
   logg = 4.52  
   MH = -0.02  
   alpha = 0.0554  
   Vmic = 1.41  
   Vmac = 5.03  
   Vsini = 2  
   limb_darkening_coeff = 0.5559810971  
   R = 48000  
Cache: 6347 4.52 -0.02 0.01 1.41 5.03 4.00 0.56 48000 loggf [] abund [] vrad []
Cache: 6347 4.52 -0.02 0.01 1.41 5.03 24.96 0.56 48000 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  13756.24815  DOF =  8652
   teff = 6347  
   logg = 4.52  
   MH = -0.02  
   alpha = 0.0554  
   Vmic = 1.41  
   Vmac = 5.03  
   Vsini = 24.96408679  
   limb_darkening_coeff = 0.5559810971  
   R = 48000  
Cache: 6347 4.52 -0.02 0.01 1.41 5.03 26.96 0.56 48000 loggf [] abund [] vrad []
Cache: 6347 4.52 -0.02 0.01 1.41 5.03 42.04 0.56 48000 loggf [] abund [] vrad []
Iter       3    CHI-SQUARE =  10358.84722  DOF =  8652
   teff = 6347  
   logg = 4.52  
   MH = -0.02  
   alpha = 0.0554

[2023-07-27 19:43:54,472] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 19:43:54,474] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    0.74	  0.0323	    0.01	  0.0004	  0.0188
Line        487.21	  487.20	  487.26	   42.15	  0.7145	    0.53	  0.0090	  0.0942
Line        489.15	  489.12	  489.21	  195.07	  2.0976	    2.41	  0.0259	  0.1601
Line        489.38	  489.36	  489.40	    0.21	  0.0058	    0.00	  0.0001	  0.0082
Line        490.33	  490.28	  490.37	   33.29	  0.4060	    0.40	  0.0048	  0.0690
Line        492.05	  492.00	  492.09	  531.03	  6.0344	    3.62	  0.0412	  0.2018
Line        492.39	  492.34	  492.42	   49.36	  0.6495	    0.53	  0.0070	  0.0830
Line        492.48	  492.45	  492.51	   27.70	  0.4540	    0.27	  0.0044	  0.0657
Line        493.88	  493.86	  493.91	   55.95	  1.1418	    0.56	  0.0115	  0.1062
Line        493.97	  493.94	  493.99	   22.05	  0.4900	    0.22	  0.0049	  0.0695
Line        496.26	  496.24	  496.28	   17.21	  0.4650	    0.14	  0.0038	  0.0611
Line        

169

# WASP-87A


## Turbospectrum 1

In [21]:
line_list = "1"
code = "turbospectrum"
star_name = 'WASP-87A'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

58


[2023-07-27 18:19:35,381] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 18:19:35,387] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 18:19:35,392] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 18:19:36,048] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 18:19:36,860] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 18:19:37,545] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 18:19:38,210] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 18:19:38,888] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 18:19:39,577] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 18:19:40,243] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 18:19:40,908] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 18:19:41,583] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 18:19:42,268] [INFO] [spectrum:estimate_snr:458

Generating: 6529 4.51 -0.28 0.11 2.10 5.95 2.00 0.54 107200 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  91285.40349  DOF =  8653
   teff = 6529  
   logg = 4.51  
   MH = -0.28  
   alpha = 0.1256  
   Vmic = 2.1  
   Vmac = 5.95  
   Vsini = 2  
   limb_darkening_coeff = 0.5361850794  
   R = 107200  
Cache: 6529 4.51 -0.28 0.11 2.10 5.95 4.00 0.54 107200 loggf [] abund [] vrad []
Cache: 6529 4.51 -0.28 0.11 2.10 5.95 21.65 0.54 107200 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  33863.84312  DOF =  8653
   teff = 6529  
   logg = 4.51  
   MH = -0.28  
   alpha = 0.1256  
   Vmic = 2.1  
   Vmac = 5.95  
   Vsini = 21.64737354  
   limb_darkening_coeff = 0.5361850794  
   R = 107200  
Cache: 6529 4.51 -0.28 0.11 2.10 5.95 23.65 0.54 107200 loggf [] abund [] vrad []
Cache: 6529 4.51 -0.28 0.11 2.10 5.95 36.04 0.54 107200 loggf [] abund [] vrad []
Iter       3    CHI-SQUARE =  23202.19416  DOF =  8653
   teff = 6529  
   logg = 4.51  
   MH = -0.28  
   alpha = 0

[2023-07-27 18:22:26,376] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 18:22:26,378] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    5.37	  0.2439	    0.55	  0.0249	  0.1545
Line        487.21	  487.20	  487.26	   35.90	  0.5984	    2.11	  0.0351	  0.1859
Line        489.15	  489.12	  489.21	   44.29	  0.4762	    1.68	  0.0180	  0.1336
Line        489.38	  489.36	  489.40	    0.28	  0.0077	    0.01	  0.0003	  0.0163
Line        490.33	  490.28	  490.37	    4.26	  0.0520	    0.13	  0.0016	  0.0393
Line        492.05	  492.00	  492.09	  487.85	  5.4815	    4.60	  0.0517	  0.2260
Line        492.39	  492.34	  492.42	   13.28	  0.1748	    0.28	  0.0037	  0.0605
Line        492.48	  492.45	  492.51	    6.62	  0.1086	    0.14	  0.0022	  0.0469
Line        493.88	  493.86	  493.91	   30.12	  0.6275	    0.52	  0.0109	  0.1035
Line        493.97	  493.94	  493.99	   25.66	  0.5702	    0.45	  0.0099	  0.0986
Line        496.26	  496.24	  496.28	  146.01	  3.9462	    1.35	  0.0364	  0.1882
Line        

179

## Synthe 1

In [31]:
line_list = "1"
code = "synthe"
star_name = 'WASP-87A'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

[2023-07-27 19:11:43,867] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 19:11:43,873] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 19:11:43,877] [INFO] [spectrum:estimate_snr:451]: 0.00%


58


[2023-07-27 19:11:44,687] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 19:11:45,726] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 19:11:46,504] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 19:11:47,197] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 19:11:47,911] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 19:11:48,598] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 19:11:49,284] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 19:11:49,956] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 19:11:50,634] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 19:11:51,337] [INFO] [spectrum:estimate_snr:458]: SNR = 149.40
[2023-07-27 19:11:51,438] [INFO] [continuum:__fit_continuum:637]: Resampling spectrum to wave_step: 0.00100 nm (R ~ 472928)
[2023-07-27 19:11:54,675] [INFO] [continuum:__median_filter:307]: Median filter with step: 51
[2023-07-27 19:11:54,847] [INFO] [continuum:__max_filter:298]: Max filter with step: 4

Generating: 6529 4.51 -0.28 0.11 2.10 5.95 2.00 0.54 107200 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  88236.15036  DOF =  8663
   teff = 6529  
   logg = 4.51  
   MH = -0.28  
   alpha = 0.1256  
   Vmic = 2.1  
   Vmac = 5.95  
   Vsini = 2  
   limb_darkening_coeff = 0.5361850794  
   R = 107200  
Cache: 6529 4.51 -0.28 0.11 2.10 5.95 4.00 0.54 107200 loggf [] abund [] vrad []
Cache: 6529 4.51 -0.28 0.11 2.10 5.95 21.63 0.54 107200 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  32821.28352  DOF =  8663
   teff = 6529  
   logg = 4.51  
   MH = -0.28  
   alpha = 0.1256  
   Vmic = 2.1  
   Vmac = 5.95  
   Vsini = 21.62886862  
   limb_darkening_coeff = 0.5361850794  
   R = 107200  
Cache: 6529 4.51 -0.28 0.11 2.10 5.95 23.63 0.54 107200 loggf [] abund [] vrad []
Cache: 6529 4.51 -0.28 0.11 2.10 5.95 36.01 0.54 107200 loggf [] abund [] vrad []
Iter       3    CHI-SQUARE =  22564.08861  DOF =  8663
   teff = 6529  
   logg = 4.51  
   MH = -0.28  
   alpha = 0

[2023-07-27 19:15:18,346] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 19:15:18,355] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    5.66	  0.2572	    0.58	  0.0263	  0.1585
Line        487.21	  487.20	  487.26	   33.55	  0.5687	    1.96	  0.0332	  0.1807
Line        489.15	  489.12	  489.21	   43.17	  0.4642	    1.63	  0.0175	  0.1316
Line        489.38	  489.36	  489.40	    0.32	  0.0089	    0.01	  0.0003	  0.0174
Line        490.33	  490.28	  490.37	    4.24	  0.0517	    0.13	  0.0015	  0.0391
Line        492.05	  492.00	  492.09	  492.63	  5.5352	    4.62	  0.0519	  0.2265
Line        492.39	  492.34	  492.42	   12.97	  0.1707	    0.27	  0.0036	  0.0596
Line        492.48	  492.45	  492.51	    6.34	  0.1039	    0.13	  0.0021	  0.0458
Line        493.88	  493.86	  493.91	   30.12	  0.6274	    0.52	  0.0109	  0.1033
Line        493.97	  493.94	  493.99	   26.08	  0.5671	    0.45	  0.0098	  0.0982
Line        496.26	  496.24	  496.28	  146.34	  3.8511	    1.35	  0.0355	  0.1861
Line        

172

## Moog 1

In [40]:
line_list = "1"
code = "moog"
star_name = 'WASP-87A'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
  

58


[2023-07-27 19:43:54,628] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 19:43:54,633] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 19:43:54,635] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 19:43:55,254] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 19:43:55,865] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 19:43:56,496] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 19:43:57,131] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 19:43:57,818] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 19:43:58,406] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 19:43:58,995] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 19:43:59,574] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 19:44:00,336] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 19:44:00,929] [INFO] [spectrum:estimate_snr:458

Generating: 6529 4.51 -0.28 0.11 2.10 5.95 2.00 0.54 107200 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  91789.70148  DOF =  8655
   teff = 6529  
   logg = 4.51  
   MH = -0.28  
   alpha = 0.1256  
   Vmic = 2.1  
   Vmac = 5.95  
   Vsini = 2  
   limb_darkening_coeff = 0.5361850794  
   R = 107200  
Cache: 6529 4.51 -0.28 0.11 2.10 5.95 4.00 0.54 107200 loggf [] abund [] vrad []
Cache: 6529 4.51 -0.28 0.11 2.10 5.95 21.60 0.54 107200 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  33882.86093  DOF =  8655
   teff = 6529  
   logg = 4.51  
   MH = -0.28  
   alpha = 0.1256  
   Vmic = 2.1  
   Vmac = 5.95  
   Vsini = 21.59663775  
   limb_darkening_coeff = 0.5361850794  
   R = 107200  
Cache: 6529 4.51 -0.28 0.11 2.10 5.95 23.60 0.54 107200 loggf [] abund [] vrad []
Cache: 6529 4.51 -0.28 0.11 2.10 5.95 35.86 0.54 107200 loggf [] abund [] vrad []
Iter       3    CHI-SQUARE =  23254.91689  DOF =  8655
   teff = 6529  
   logg = 4.51  
   MH = -0.28  
   alpha = 0

[2023-07-27 19:44:48,129] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 19:44:48,132] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    5.57	  0.2531	    0.57	  0.0259	  0.1574
Line        487.21	  487.20	  487.26	   18.55	  0.3091	    1.09	  0.0182	  0.1337
Line        489.15	  489.12	  489.21	   37.69	  0.4053	    1.43	  0.0153	  0.1232
Line        489.38	  489.36	  489.40	    0.74	  0.0204	    0.03	  0.0007	  0.0265
Line        490.33	  490.28	  490.37	    3.17	  0.0386	    0.10	  0.0012	  0.0339
Line        492.05	  492.00	  492.09	  493.95	  5.5500	    4.65	  0.0523	  0.2273
Line        492.39	  492.34	  492.42	   10.92	  0.1437	    0.23	  0.0030	  0.0549
Line        492.48	  492.45	  492.51	    5.97	  0.0963	    0.12	  0.0020	  0.0441
Line        493.88	  493.86	  493.91	   30.17	  0.6285	    0.53	  0.0110	  0.1036
Line        493.97	  493.94	  493.99	   25.41	  0.5647	    0.44	  0.0098	  0.0981
Line        496.26	  496.24	  496.28	  145.78	  3.9400	    1.35	  0.0364	  0.1882
Line        

171

# HATS-10

## Turbospectrum 1

In [23]:
line_list = "1"
code = "turbospectrum"
star_name = 'HATS-10'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

48


[2023-07-27 18:38:22,183] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 18:38:22,198] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 18:38:22,207] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 18:38:22,786] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 18:38:23,341] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 18:38:23,856] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 18:38:24,403] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 18:38:24,941] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 18:38:25,502] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 18:38:26,065] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 18:38:26,611] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 18:38:27,157] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 18:38:27,679] [INFO] [spectrum:estimate_snr:458

Generating: 6167 4.51 0.40 0.00 1.45 4.28 2.00 0.59 48000 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  7076.092096  DOF =  8648
   teff = 6167  
   logg = 4.51  
   MH = 0.4  
   alpha = -0.058  
   Vmic = 1.45  
   Vmac = 4.28  
   Vsini = 2  
   limb_darkening_coeff = 0.5932992429  
   R = 48000  
Cache: 6167 4.51 0.40 0.00 1.45 4.28 4.00 0.59 48000 loggf [] abund [] vrad []
Cache: 6167 4.51 0.40 0.00 1.45 4.28 5.08 0.59 48000 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  7020.885144  DOF =  8648
   teff = 6167  
   logg = 4.51  
   MH = 0.4  
   alpha = -0.058  
   Vmic = 1.45  
   Vmac = 4.28  
   Vsini = 5.078548351  
   limb_darkening_coeff = 0.5932992429  
   R = 48000  
Cache: 6167 4.51 0.40 0.00 1.45 4.28 7.08 0.59 48000 loggf [] abund [] vrad []
Cache: 6167 4.51 0.40 0.00 1.45 4.28 4.82 0.59 48000 loggf [] abund [] vrad []
Iter       3    CHI-SQUARE =  7019.954541  DOF =  8648
   teff = 6167  
   logg = 4.51  
   MH = 0.4  
   alpha = -0.058  
   Vmic = 1

[2023-07-27 18:41:16,478] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 18:41:16,481] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    1.11	  0.0484	    0.10	  0.0045	  0.0654
Line        487.21	  487.20	  487.26	    1.58	  0.0263	    0.10	  0.0016	  0.0402
Line        489.15	  489.12	  489.21	    4.36	  0.0469	    0.29	  0.0031	  0.0551
Line        489.38	  489.36	  489.40	    1.24	  0.0345	    0.10	  0.0029	  0.0531
Line        490.33	  490.28	  490.37	    3.57	  0.0435	    0.27	  0.0032	  0.0566
Line        492.05	  492.00	  492.09	    4.91	  0.0552	    0.33	  0.0037	  0.0603
Line        492.39	  492.34	  492.42	    2.20	  0.0289	    0.15	  0.0019	  0.0437
Line        492.48	  492.45	  492.51	    3.06	  0.0502	    0.23	  0.0038	  0.0612
Line        493.88	  493.86	  493.91	    2.15	  0.0449	    0.14	  0.0029	  0.0535
Line        493.97	  493.94	  493.99	    2.74	  0.0610	    0.19	  0.0042	  0.0640
Line        496.26	  496.24	  496.28	    1.40	  0.0368	    0.10	  0.0027	  0.0512
Line        

164

## Synthe 1

In [32]:
line_list = "1"
code = "synthe"
star_name = 'HATS-10'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

48


[2023-07-27 19:15:18,833] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 19:15:18,847] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 19:15:18,851] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 19:15:19,831] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 19:15:20,618] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 19:15:21,298] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 19:15:21,960] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 19:15:22,634] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 19:15:23,296] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 19:15:23,957] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 19:15:24,629] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 19:15:25,340] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 19:15:26,009] [INFO] [spectrum:estimate_snr:458

Generating: 6167 4.51 0.40 0.00 1.45 4.28 2.00 0.59 48000 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  7101.166206  DOF =  8648
   teff = 6167  
   logg = 4.51  
   MH = 0.4  
   alpha = -0.058  
   Vmic = 1.45  
   Vmac = 4.28  
   Vsini = 2  
   limb_darkening_coeff = 0.5932992429  
   R = 48000  
Cache: 6167 4.51 0.40 0.00 1.45 4.28 4.00 0.59 48000 loggf [] abund [] vrad []
Cache: 6167 4.51 0.40 0.00 1.45 4.28 5.16 0.59 48000 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  7042.289023  DOF =  8648
   teff = 6167  
   logg = 4.51  
   MH = 0.4  
   alpha = -0.058  
   Vmic = 1.45  
   Vmac = 4.28  
   Vsini = 5.161126791  
   limb_darkening_coeff = 0.5932992429  
   R = 48000  
Cache: 6167 4.51 0.40 0.00 1.45 4.28 7.16 0.59 48000 loggf [] abund [] vrad []
Cache: 6167 4.51 0.40 0.00 1.45 4.28 4.89 0.59 48000 loggf [] abund [] vrad []
Iter       3    CHI-SQUARE =  7041.224834  DOF =  8648
   teff = 6167  
   logg = 4.51  
   MH = 0.4  
   alpha = -0.058  
   Vmic = 1

[2023-07-27 19:18:51,224] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 19:18:51,236] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    0.80	  0.0350	    0.07	  0.0032	  0.0553
Line        487.21	  487.20	  487.26	    1.40	  0.0233	    0.09	  0.0016	  0.0392
Line        489.15	  489.12	  489.21	    3.90	  0.0419	    0.25	  0.0027	  0.0517
Line        489.38	  489.36	  489.40	    1.22	  0.0338	    0.10	  0.0028	  0.0518
Line        490.33	  490.28	  490.37	    3.22	  0.0392	    0.23	  0.0028	  0.0524
Line        492.05	  492.00	  492.09	    3.79	  0.0426	    0.25	  0.0028	  0.0524
Line        492.39	  492.34	  492.42	    2.60	  0.0343	    0.17	  0.0022	  0.0470
Line        492.48	  492.45	  492.51	    2.98	  0.0489	    0.23	  0.0037	  0.0604
Line        493.88	  493.86	  493.91	    1.74	  0.0362	    0.11	  0.0023	  0.0478
Line        493.97	  493.94	  493.99	    3.20	  0.0711	    0.22	  0.0048	  0.0686
Line        496.26	  496.24	  496.28	    0.96	  0.0252	    0.07	  0.0018	  0.0416
Line        

156

## Moog 1

In [41]:
line_list = "1"
code = "moog"
star_name = 'HATS-10'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

48


[2023-07-27 19:44:48,476] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 19:44:48,483] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 19:44:48,486] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 19:44:49,143] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 19:44:49,834] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 19:44:50,524] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 19:44:51,151] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 19:44:51,825] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 19:44:52,483] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 19:44:53,095] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 19:44:53,764] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 19:44:54,445] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 19:44:55,145] [INFO] [spectrum:estimate_snr:458

Generating: 6167 4.51 0.40 0.00 1.45 4.28 2.00 0.59 48000 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  7186.454196  DOF =  8648
   teff = 6167  
   logg = 4.51  
   MH = 0.4  
   alpha = -0.058  
   Vmic = 1.45  
   Vmac = 4.28  
   Vsini = 2  
   limb_darkening_coeff = 0.5932992429  
   R = 48000  
Cache: 6167 4.51 0.40 0.00 1.45 4.28 4.00 0.59 48000 loggf [] abund [] vrad []
Cache: 6167 4.51 0.40 0.00 1.45 4.28 5.17 0.59 48000 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  7125.873473  DOF =  8648
   teff = 6167  
   logg = 4.51  
   MH = 0.4  
   alpha = -0.058  
   Vmic = 1.45  
   Vmac = 4.28  
   Vsini = 5.169508936  
   limb_darkening_coeff = 0.5932992429  
   R = 48000  
Cache: 6167 4.51 0.40 0.00 1.45 4.28 7.17 0.59 48000 loggf [] abund [] vrad []
Cache: 6167 4.51 0.40 0.00 1.45 4.28 4.89 0.59 48000 loggf [] abund [] vrad []
Iter       3    CHI-SQUARE =  7124.736389  DOF =  8648
   teff = 6167  
   logg = 4.51  
   MH = 0.4  
   alpha = -0.058  
   Vmic = 1

[2023-07-27 19:45:43,132] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 19:45:43,136] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    0.87	  0.0380	    0.08	  0.0035	  0.0580
Line        487.21	  487.20	  487.26	    2.16	  0.0360	    0.15	  0.0024	  0.0489
Line        489.15	  489.12	  489.21	    4.85	  0.0522	    0.33	  0.0035	  0.0592
Line        489.38	  489.36	  489.40	    1.28	  0.0355	    0.10	  0.0029	  0.0529
Line        490.33	  490.28	  490.37	    2.89	  0.0353	    0.21	  0.0025	  0.0501
Line        492.05	  492.00	  492.09	    4.07	  0.0463	    0.25	  0.0028	  0.0527
Line        492.39	  492.34	  492.42	    1.87	  0.0246	    0.13	  0.0018	  0.0419
Line        492.48	  492.45	  492.51	    4.28	  0.0702	    0.32	  0.0052	  0.0717
Line        493.88	  493.86	  493.91	    1.85	  0.0386	    0.13	  0.0027	  0.0511
Line        493.97	  493.94	  493.99	    4.07	  0.0905	    0.28	  0.0062	  0.0781
Line        496.26	  496.24	  496.28	    1.20	  0.0317	    0.09	  0.0024	  0.0486
Line        

152

# HATS-56

## Turbospectrum 1

In [24]:
line_list = "1"
code = "turbospectrum"
star_name = 'HATS-56'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

71


[2023-07-27 18:41:16,711] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 18:41:16,718] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 18:41:16,721] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 18:41:17,368] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 18:41:18,012] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 18:41:18,610] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 18:41:19,300] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 18:41:19,946] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 18:41:20,583] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 18:41:21,206] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 18:41:21,790] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 18:41:22,368] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 18:41:22,947] [INFO] [spectrum:estimate_snr:458

Generating: 6594 3.98 0.25 0.00 1.75 7.37 2.00 0.55 115000 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  4166.025704  DOF =  8652
   teff = 6594  
   logg = 3.98  
   MH = 0.25  
   alpha = -0.0175  
   Vmic = 1.75  
   Vmac = 7.37  
   Vsini = 2  
   limb_darkening_coeff = 0.5480774879  
   R = 115000  
Cache: 6594 3.98 0.25 0.00 1.75 7.37 4.00 0.55 115000 loggf [] abund [] vrad []
Cache: 6594 3.98 0.25 0.00 1.75 7.37 1.24 0.55 115000 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  4123.889746  DOF =  8652
   teff = 6594  
   logg = 3.98  
   MH = 0.25  
   alpha = -0.0175  
   Vmic = 1.75  
   Vmac = 7.37  
   Vsini = 1.244233353  
   limb_darkening_coeff = 0.5480774879  
   R = 115000  
Cache: 6594 3.98 0.25 0.00 1.75 7.37 3.24 0.55 115000 loggf [] abund [] vrad []
Cache: 6594 3.98 0.25 0.00 1.75 7.37 0.81 0.55 115000 loggf [] abund [] vrad []
Iter       3    CHI-SQUARE =  4116.055894  DOF =  8652
   teff = 6594  
   logg = 3.98  
   MH = 0.25  
   alpha = -0.0175 

[2023-07-27 18:44:22,044] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 18:44:22,045] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    4.67	  0.2123	    0.02	  0.0009	  0.0301
Line        487.21	  487.20	  487.26	   21.86	  0.3704	    0.13	  0.0023	  0.0472
Line        489.15	  489.12	  489.21	   33.31	  0.3621	    0.23	  0.0025	  0.0493
Line        489.38	  489.36	  489.40	   11.49	  0.3192	    0.09	  0.0025	  0.0496
Line        490.33	  490.28	  490.37	   21.30	  0.2597	    0.14	  0.0017	  0.0412
Line        492.05	  492.00	  492.09	   34.92	  0.3969	    0.17	  0.0020	  0.0441
Line        492.39	  492.34	  492.42	   52.32	  0.6884	    0.22	  0.0029	  0.0531
Line        492.48	  492.45	  492.51	   11.48	  0.1882	    0.08	  0.0012	  0.0350
Line        493.88	  493.86	  493.91	    9.81	  0.2044	    0.06	  0.0012	  0.0340
Line        493.97	  493.94	  493.99	   12.78	  0.2840	    0.07	  0.0016	  0.0399
Line        496.26	  496.24	  496.28	    7.21	  0.1897	    0.04	  0.0012	  0.0338
Line        

164

## Synthe 1

In [33]:
line_list = "1"
code = "synthe"
star_name = 'HATS-56'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
   

71


[2023-07-27 19:18:51,482] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 19:18:51,488] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 19:18:51,492] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 19:18:52,246] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 19:18:53,072] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 19:18:53,722] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 19:18:54,393] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 19:18:55,058] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 19:18:55,721] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 19:18:56,388] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 19:18:57,052] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 19:18:57,733] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 19:18:58,448] [INFO] [spectrum:estimate_snr:458

Generating: 6594 3.98 0.25 0.00 1.75 7.37 2.00 0.55 115000 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  4236.58024  DOF =  8652
   teff = 6594  
   logg = 3.98  
   MH = 0.25  
   alpha = -0.0175  
   Vmic = 1.75  
   Vmac = 7.37  
   Vsini = 2  
   limb_darkening_coeff = 0.5480774879  
   R = 115000  
Cache: 6594 3.98 0.25 0.00 1.75 7.37 4.00 0.55 115000 loggf [] abund [] vrad []
Cache: 6594 3.98 0.25 0.00 1.75 7.37 1.48 0.55 115000 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  4214.299452  DOF =  8652
   teff = 6594  
   logg = 3.98  
   MH = 0.25  
   alpha = -0.0175  
   Vmic = 1.75  
   Vmac = 7.37  
   Vsini = 1.476682595  
   limb_darkening_coeff = 0.5480774879  
   R = 115000  
Cache: 6594 3.98 0.25 0.00 1.75 7.37 3.48 0.55 115000 loggf [] abund [] vrad []
Cache: 6594 3.98 0.25 0.00 1.75 7.37 1.21 0.55 115000 loggf [] abund [] vrad []
Iter       3    CHI-SQUARE =  4209.913453  DOF =  8652
   teff = 6594  
   logg = 3.98  
   MH = 0.25  
   alpha = -0.0175  

[2023-07-27 19:22:17,280] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 19:22:17,292] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    4.90	  0.2228	    0.02	  0.0010	  0.0307
Line        487.21	  487.20	  487.26	   21.39	  0.3626	    0.13	  0.0022	  0.0466
Line        489.15	  489.12	  489.21	   32.37	  0.3519	    0.22	  0.0024	  0.0484
Line        489.38	  489.36	  489.40	   11.56	  0.3210	    0.09	  0.0025	  0.0491
Line        490.33	  490.28	  490.37	   20.57	  0.2508	    0.13	  0.0016	  0.0401
Line        492.05	  492.00	  492.09	   28.45	  0.3232	    0.15	  0.0017	  0.0404
Line        492.39	  492.34	  492.42	   66.20	  0.8710	    0.26	  0.0034	  0.0583
Line        492.48	  492.45	  492.51	   10.67	  0.1749	    0.07	  0.0011	  0.0330
Line        493.88	  493.86	  493.91	   13.95	  0.2905	    0.07	  0.0016	  0.0391
Line        493.97	  493.94	  493.99	   15.54	  0.3453	    0.08	  0.0019	  0.0428
Line        496.26	  496.24	  496.28	    7.48	  0.1967	    0.04	  0.0012	  0.0336
Line        

156

## Moog 1

In [42]:
line_list = "1"
code = "moog"
star_name = 'HATS-56'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
   

71


[2023-07-27 19:45:43,376] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 19:45:43,382] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 19:45:43,385] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 19:45:43,996] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 19:45:44,647] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 19:45:45,262] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 19:45:45,830] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 19:45:46,403] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 19:45:46,973] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 19:45:47,542] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 19:45:48,114] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 19:45:48,706] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 19:45:49,311] [INFO] [spectrum:estimate_snr:458

Generating: 6594 3.98 0.25 0.00 1.75 7.37 2.00 0.55 115000 loggf [] abund [] vrad []
Iter       1    CHI-SQUARE =  4256.800289  DOF =  8652
   teff = 6594  
   logg = 3.98  
   MH = 0.25  
   alpha = -0.0175  
   Vmic = 1.75  
   Vmac = 7.37  
   Vsini = 2  
   limb_darkening_coeff = 0.5480774879  
   R = 115000  
Cache: 6594 3.98 0.25 0.00 1.75 7.37 4.00 0.55 115000 loggf [] abund [] vrad []
Cache: 6594 3.98 0.25 0.00 1.75 7.37 1.79 0.55 115000 loggf [] abund [] vrad []
Iter       2    CHI-SQUARE =  4252.506081  DOF =  8652
   teff = 6594  
   logg = 3.98  
   MH = 0.25  
   alpha = -0.0175  
   Vmic = 1.75  
   Vmac = 7.37  
   Vsini = 1.791526096  
   limb_darkening_coeff = 0.5480774879  
   R = 115000  
Cache: 6594 3.98 0.25 0.00 1.75 7.37 3.79 0.55 115000 loggf [] abund [] vrad []
Cache: 6594 3.98 0.25 0.00 1.75 7.37 1.71 0.55 115000 loggf [] abund [] vrad []
Iter       3    CHI-SQUARE =  4251.639573  DOF =  8652
   teff = 6594  
   logg = 3.98  
   MH = 0.25  
   alpha = -0.0175 

[2023-07-27 19:46:32,548] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:228]: Saving results...
[2023-07-27 19:46:32,549] [INFO] [automatic_calculated_limb_calculated_alpha:determine_astrophysical_parameters_using_synth_spectra:237]: Saving synthetic spectrum...




          wave_peak	wave_base	wave_top	  wchisq	 rwchisq	   chisq	  rchisq	     rms
Line        480.29	  480.29	  480.31	    4.45	  0.2021	    0.02	  0.0009	  0.0301
Line        487.21	  487.20	  487.26	   31.51	  0.5341	    0.21	  0.0036	  0.0591
Line        489.15	  489.12	  489.21	   31.89	  0.3467	    0.22	  0.0024	  0.0487
Line        489.38	  489.36	  489.40	    8.96	  0.2489	    0.07	  0.0020	  0.0440
Line        490.33	  490.28	  490.37	   22.00	  0.2683	    0.14	  0.0017	  0.0416
Line        492.05	  492.00	  492.09	   35.72	  0.4059	    0.17	  0.0020	  0.0441
Line        492.39	  492.34	  492.42	  108.48	  1.4273	    0.47	  0.0062	  0.0781
Line        492.48	  492.45	  492.51	   10.64	  0.1744	    0.07	  0.0011	  0.0330
Line        493.88	  493.86	  493.91	   10.82	  0.2255	    0.06	  0.0013	  0.0352
Line        493.97	  493.94	  493.99	   13.26	  0.2947	    0.07	  0.0016	  0.0400
Line        496.26	  496.24	  496.28	    7.67	  0.2019	    0.05	  0.0012	  0.0342
Line        

153

# KPS-1

## Turbospectrum 1

In [ ]:
line_list = "1"
code = "turbospectrum"
star_name = 'KPS-1'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

## Synthe 1

## Moog 1

# Kepler-40

## Turbospectrum 1

In [22]:
line_list = "1"
code = "turbospectrum"
star_name = 'Kepler-40'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

76


[2023-07-27 18:22:26,579] [INFO] [automatic_calculated_limb_calculated_alpha:cut_spectrum_from_range:45]: Cutting...
[2023-07-27 18:22:26,586] [INFO] [automatic_calculated_limb_calculated_alpha:estimate_snr_from_flux:56]: Estimating SNR from fluxes...
[2023-07-27 18:22:26,589] [INFO] [spectrum:estimate_snr:451]: 0.00%
[2023-07-27 18:22:27,207] [INFO] [spectrum:estimate_snr:451]: 10.00%
[2023-07-27 18:22:27,821] [INFO] [spectrum:estimate_snr:451]: 20.00%
[2023-07-27 18:22:28,462] [INFO] [spectrum:estimate_snr:451]: 30.00%
[2023-07-27 18:22:29,036] [INFO] [spectrum:estimate_snr:451]: 40.00%
[2023-07-27 18:22:29,604] [INFO] [spectrum:estimate_snr:451]: 50.00%
[2023-07-27 18:22:30,225] [INFO] [spectrum:estimate_snr:451]: 60.00%
[2023-07-27 18:22:30,824] [INFO] [spectrum:estimate_snr:451]: 70.00%
[2023-07-27 18:22:31,456] [INFO] [spectrum:estimate_snr:451]: 80.00%
[2023-07-27 18:22:32,096] [INFO] [spectrum:estimate_snr:451]: 90.00%
[2023-07-27 18:22:32,777] [INFO] [spectrum:estimate_snr:458

IndexError: index 0 is out of bounds for axis 0 with size 0

## Synthe 1

## Moog 1

# Kepler-435

## Turbospectrum 1

In [ ]:
line_list = "1"
code = "turbospectrum"
star_name = 'Kepler-435'

i = sweet_cat_parameters.loc[sweet_cat_parameters['Star']==star_name]['ind'].values[0]
print(i)
star_file = sweet_cat_parameters['Spectra'][i]
snr = sweet_cat_parameters['SNR'][i]
initial_teff = sweet_cat_parameters['teff'][i]
initial_logg = sweet_cat_parameters['logg'][i]
initial_MH = sweet_cat_parameters['MH'][i]
initial_vmic = sweet_cat_parameters['vmic'][i]
initial_R = sweet_cat_parameters['Resolution'][i]

normed_star_file, star_spectrum, estimated_snr, rv, rv_err = normalizing(path, star_file, line_list, snr, lower_lim=470, upper_lim=680)

params, errors, abundances_found, loggf_found, status, stats_linemasks = determine_astrophysical_parameters_using_synth_spectra(code, path, normed_star_file, line_list, initial_teff,initial_logg,initial_MH,initial_vmic,initial_R)

vsini = params['vsini']
err_vsini = errors['vsini']

#literatue values
vsini_lit = sweet_cat_parameters['vsini'][i]
err_up_vsini_lit = sweet_cat_parameters['err_up_vsini'][i]
err_down_vsini_lit = sweet_cat_parameters['err_down_vsini'][i]


#percent error
per_err = (np.abs(vsini - vsini_lit)/vsini_lit) * 100

#percent uncertainty
per_un = (err_vsini/vsini) * 100

print('\n\n')
print(star_name)
print('lit', vsini_lit)
print('ispec', vsini, ' +/- ', err_vsini)
print(vsini_lit+err_down_vsini_lit < vsini < vsini_lit+err_up_vsini_lit)
print(per_err)
print(per_un)
print('\n\n')

#star file
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
f = open("/home/spec/Programs/iSpec/automatization/results/{}.txt".format(star_name), "a")
f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(code, line_list,dt_string,sweet_cat_parameters['Star_SC'][i],vsini,err_vsini,per_err,per_un,rv,rv_err,params['vmac'],params['alpha'],errors['alpha'],params['limb_darkening_coeff']))
        

## Synthe 1

## Moog 1